In [8]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score

# from google.colab import files
# uploaded = files.upload()
from google.colab import drive
drive.mount('/content/drive')

# Load the dataset
data = pd.read_csv('/content/drive/My Drive/[[[[ECCE 2025 CUET]]]]/Implementation of Existing Model/online_shoppers_intention.csv')

# Define numerical and categorical columns
numerical_columns = ['Administrative', 'Administrative_Duration', 'Informational', 'Informational_Duration',
                     'ProductRelated', 'ProductRelated_Duration', 'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay']
categorical_columns = ['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType', 'Weekend']

# Separate features and target variable
X = data[numerical_columns + categorical_columns]
y = data['Revenue']  # Assuming 'Revenue' is the dependent variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Preprocess the data**

In [2]:
# Preprocess the data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(drop='first'), categorical_columns)
    ]
)

X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

**Train the Linear Regression model and make prediction**

In [3]:
# Train the Linear Regression model
model = LinearRegression()
model.fit(X_train_transformed, y_train)

# Make predictions
y_pred = model.predict(X_test_transformed)

**Evaluate the model**

In [6]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

Mean Squared Error: 0.10
R-squared: 0.28


**Feature Importance (Coefficients)**

In [5]:
# Feature Importance (Coefficients)
feature_names = (
    numerical_columns +
    preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_columns).tolist()
)

coefficients = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': model.coef_
}).sort_values(by='Coefficient', ascending=False)

print("\nFeature Importance:")
print(coefficients)


Feature Importance:
               Feature  Coefficient
60      TrafficType_16     0.248477
36          Browser_12     0.217129
8           PageValues     0.164790
24  OperatingSystems_7     0.116591
37          Browser_13     0.089587
..                 ...          ...
11           Month_Feb    -0.060115
10           Month_Dec    -0.061241
59      TrafficType_15    -0.062744
62      TrafficType_18    -0.064890
25  OperatingSystems_8    -0.131276

[68 rows x 2 columns]


In [11]:
# Make predictions and convert to binary using a threshold (e.g., 0.5)
y_pred = model.predict(X_test_transformed)
y_pred_binary = np.where(y_pred > 0.5, 1, 0)  # Convert to 0 or 1 based on threshold

# Calculate classification metrics using the binary predictions
accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.86
Precision: 0.79
Recall: 0.24
F1 Score: 0.37
